In [17]:
using Pkg
Pkg.activate("../")
using HTTP
using JSON
using JSON3
using Dates
using PyPlot
const plt = PyPlot
using NCDatasets
using OrderedCollections
include("./DIVAndFairEase.jl")
token = ENV["beaconAPItoken"];

  Activating project at `~/Projects/DIVAnd-FAIR-EASE`


In [50]:
using Conda
Conda.add("cartopy")
using PyCall
ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
mpl = pyimport("matplotlib")
mpl.style.use("./fairease.mplstyle")
coast = cfeature.GSHHSFeature(scale="h")
datacrs = ccrs.PlateCarree();
figdir = "../figures/NorthAdriatic"
mkpath(figdir)

"../figures/NorthAdriatic"

In [3]:
datadir = "../data/"
footprintdir = "../Footprint"
mkpath.([datadir, footprintdir])

2-element Vector{String}:
 "../data"
 "../Footprint"

In [4]:
const beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://beacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://beacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
);

In [88]:
datasource = "Euro-Argo"
datasource = "World Ocean Database"
datasource = "SeaDataNet CDI TS"
datasource = "CORA Timeseries"

domain = [12., 18., 43., 46.]
parameter1 = "TEMP"
if datasource == "World Ocean Database"
    parameter1 = "Temperature"
elseif datasource == "EMODnet Chemistry"  
    parameter1 = "ITS_90_water_temperature"
elseif datasource == "SeaDataNet CDI TS"
    parameter1 = "ITS-90 water temperature"
end
parameter2 = "PSAL"

datestart = Dates.Date(1950, 1, 1)
dateend = Dates.Date(2024, 1, 1)
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 500.
regionname = "NorthAdriaticSea"
@info("$(datestart) → $(dateend), $(parameter1)")

[ Info: 1950-01-01 → 2024-01-01, TEMP


In [89]:
beaconURL = beacon_services[datasource]
footprintURL = joinpath(beaconURL, "api/datasets/footprint")
datasource_name = replace(datasource, " " => "_")
footprintfile = joinpath(footprintdir, "Footprint_$(datasource_name).json")
@info("Footprint endpoint: $(footprintURL)")
@info("Footprint file: $(footprintfile)")

[ Info: Footprint endpoint: https://beacon-cora-ts.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_CORA_Timeseries.json


In [90]:
if isfile(footprintfile)
    @info("Footprint file already downloaded")
else
    @info("Writing Footprint file")
    open(footprintfile, "w") do io
        r = HTTP.request("GET", footprintURL, 
            ["Authorization" => "Bearer $(token)"],
            response_stream=io)
        @info(r.status)
    end
end

[ Info: Footprint file already downloaded


In [91]:
# Note 1: the JSON file is 193M, just to get the list of variables?

In [92]:
data = JSON.parsefile(footprintfile);
attributes = data["unique_column_attributes"]
params = sort(collect(keys(attributes)));
parameter1 in params
# @show(params);

true

In [93]:
query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
    mindepth, maxdepth, minlon, maxlon, minlat, maxlat)

[ Info: Working with CORA dataset


"{\"query_parameters\":[{\"column_name\":\"TEMP\",\"alias\":\"TEMP\"},{\"column_name\":\"JULD\",\"alias\":\"TIME\"},{\"column_name\":\"DEPH\",\"alias\":\"DEPTH\"},{\"column_name\":\"LONGITUDE\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"LATITUDE\",\"alias\":\"LATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":0,\"max\":27028},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":500.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0},{\"for_query_parameter\":\"TEMP\",\"min\":-2.0,\"max\":30.0}],\"output\":{\"format\":\"netcdf\"}}"

In [94]:
# Note 2: cannot work with such a long list of parameters!!

In [95]:
filename = joinpath(datadir, "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
figname = "$(regionname)_$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.jpg"
filename

"../data/NorthAdriaticSea_CORA_Timeseries_TEMP_19500101-20240101_0-500m.nc"

In [96]:
@info("Data will be written in file:\n$(filename)")

if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end
@time open(filename, "w") do io
    r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
        ["Content-type"=> "application/json",
         "Authorization" => "Bearer $(token)"
        ],
        query, 
        response_stream=io);
    @info(r.status)
end

@info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")

┌ Info: Data will be written in file:
└ ../data/NorthAdriaticSea_CORA_Timeseries_TEMP_19500101-20240101_0-500m.nc


LoadError: HTTP.Exceptions.StatusError(400, "POST", "/api/query", HTTP.Messages.Response:
"""
HTTP/1.1 400 Bad Request
Content-Type: application/json
Vary: Access-Control-Request-Headers
x-beacon-query-id: 19315e6d-f456-4cd5-bf6f-b8ab64127e7a
X-Powered-By: ARR/3.0
Date: Fri, 20 Sep 2024 13:20:24 GMT
Content-Length: 94
Set-Cookie: ******

[Message Body was streamed]""")

In [97]:
## Plotting

In [98]:
NCDataset(filename, "r") do nc
    # print(nc)
    lon = nc["LONGITUDE"][:]
    lat = nc["LATITUDE"][:]

    fig = plt.figure()
    ax = plt.subplot(111, projection=ccrs.EuroPP())
    ax.plot(lon, lat, "go", ms=.5, transform=datacrs)
    ax.set_extent(domain)
    ax.add_feature(coast, lw=.25, color=".25")
    gl = ax.gridlines(crs=datacrs, draw_labels=true,
        linewidth=.5, color=".25", linestyle="--", zorder=2)
    gl.top_labels = false
    gl.right_labels = false
    ax.set_title("Temperature observations ($(length(lon))) in $(datasource)")
    plt.savefig(joinpath(figdir, figname))
    plt.close()
end

LoadError: NetCDF error: [31mOpening path ../data/NorthAdriaticSea_CORA_Timeseries_TEMP_19500101-20240101_0-500m.nc: NetCDF: Unknown file format[39m (NetCDF error code: -51)